# End of week 1 exercise

## Original

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

## My modification
To demonstrate your familiarity with OpenAI API, and also Ollama, implement a system where two models are interacting 
with each other. One model ask a Python-related question, while the other tries to answer that question.

In [ ]:
# imports
import os
from openai import OpenAI
from IPython.display import Markdown, update_display
from dotenv import load_dotenv, find_dotenv
from openai.types.chat import ChatCompletion

In [ ]:
# constants & env. setup

MODEL_GPT = 'gpt-4.1-mini'
MODEL_LLAMA = 'codellama:latest'

load_dotenv(find_dotenv(), override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
# set up models
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key='ollama')
openai = OpenAI()

def model_response(model_obj: OpenAI, model_name: str, prompt: dict, streaming: bool = False) -> ChatCompletion:
    """Generate a response from a model."""
    response = model_obj.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": prompt['system']},
            {"role": "user", "content": prompt['user']}
            ],
        stream=streaming
    )
    return response

def display_streaming(response) -> None:
    """Display a streaming response.""" 
    display_handle = display(Markdown(""), display_id=True)
    data = ""
    for chunk in response:
        data += chunk.choices[0].delta.content or ''
        update_display(Markdown(data), display_id=display_handle.display_id)

def interact(model_objects: list[OpenAI], model_names: list[str], prompts: dict) -> None:
    """Model-to-model interaction."""
    generator, answerer = model_objects
    generator_name, answerer_name = model_names
    question = model_response(generator, generator_name, prompts['question_generator']).choices[0].message.content
    print('Question:')
    display(Markdown(question))

    print('Answer:')
    prompt = prompts['question_solver'].copy()
    prompt['user'] = prompt['user'].format(question=question)
    answer = model_response(answerer, answerer_name, prompt, streaming=True)
    display_streaming(answer)


In [ ]:
# here is the question; type over this to ask something new
questiongen_system_prompt = """
You are a query generating assistant who create simple question about Python3 language.
The intermediate level Python developer should be able to answer your question.

Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.

Example questions:
Q: What does the code do: 
```
yield from {book.get("author") for book in books if book.get("author")}
```

Q: What is the purpose of the following code:
```
def greet(name):
    return f"Hello, {name}!"
```
"""

questiongen_user_prompt = "Generate an intermediate level question about Python3 language."

questionsolver_system_prompt = """You are a question solver assistant. Your task is to answer a question about Python3 language given to you by the user.
Respond in a concise manner, in a few sentences. Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.

Example:
Q: What is the purpose of the following code:
```
def greet(name):
    return f"Hello, {name}!"
```
A: This function greets the user with a message containing the user's name."""

questionsolver_user_prompt = """Provide answer to the following question:

# {question}
"""


prompts = {
    "question_generator": {
        "system": questiongen_system_prompt,
        "user": questiongen_user_prompt
    },
    "question_solver": {
        "system": questionsolver_system_prompt,
        "user": questionsolver_user_prompt
    }
}

In [ ]:
# Get gpt-4o-mini to answer, with streaming
interact([openai, openai], [MODEL_GPT, MODEL_GPT], prompts)

In [ ]:
# Get codellama to answer
interact([ollama, ollama], [MODEL_LLAMA, MODEL_LLAMA], prompts)

In [ ]:
# One model asking another model
interact([openai, ollama], [MODEL_GPT, MODEL_LLAMA], prompts)

In [ ]:
# And the other way around
interact([ollama, openai], [MODEL_LLAMA, MODEL_GPT], prompts)